# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

Now that the Retail Demo Store has trained models for 3 different use cases (Recommended for You, Customers who viewed X also viewed and Ranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer.ipynb`](02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with Recommenders and Campaigns and get recommendations. 

We will create and interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building  in this notebook highlighted in blue with a dashed outline.

![Workflow](images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [80]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [37]:
#retrieves previously stored variables 
%store -r 

In [38]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Adding some helper functions to make results more readable

First, let's create a supporting function to help make sense of the results returned by a Personalize recommender or campaign. Personalize returns only an `item_id`, and potentially information about whether that item was part of a promotion filter. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the synthetic dataset.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: If you want to lookup information as part of the API request, consider implementing Real-Time Personalization APIs - https://github.com/aws-samples/personalization-apis.
</div>



In [39]:
def get_item_name_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['name'].values[0]
    return item_name

In [40]:
def get_item_category_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['category'].values[0]
    return item_name

In [41]:
def get_item_style_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['style'].values[0]
    return item_name

Great! Now we have a way of rendering results. 

## Interact with Recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "Customers who viewed X also viewed" Recommender

"Customers who viewed X also viewed" requires an item and a user as input, and it will return customers also viewed based on an item that you specify.

The cells below will handle getting recommendations from the "Customers who viewed X also viewed" Recommender and rendering the results. Let's see what the recommendations are for an item.

We will be using the `recommenderArn`, the `itemId`, the `userId`, as well as the number or results we want, `numResults`.

#### Select a User

We'll just pick a random user for simplicity (there are about 6,000 users with user IDs assigned in sequential order). Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [42]:
user_id = 555
user_metadata_df[user_metadata_df['id']==user_id]

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
554,555,True,F,Natalie,Smith,natalie.smith@example.com,51,Natalie Smith,user555,outdoors_instruments_groceries,lower_priced_products,{},{'ios': {'anonymous_id': '6860933c-dce7-4ef0-b...,"[{'first_name': 'Natalie', 'last_name': 'Smith..."


We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [43]:
product_id = item_metadata_df.sample(1)["id"].values[0]

print ('productId: ', product_id)

productId:  18ecf2a3-198d-4994-afba-dee116bd0a41


First lets get the response directly from the get_recommendations API, which by default returns 25 items, but can be adjusted.

In [44]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_customers_who_viewed_arn,
    itemId = str(product_id),
    userId = str(user_id),
    numResults = 20
)

In [45]:
print (get_recommendations_response)

{'ResponseMetadata': {'RequestId': 'fd46a5c9-0353-494a-a5d9-d0b43fa5b698', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 27 Apr 2024 20:32:31 GMT', 'content-type': 'application/json', 'content-length': '2736', 'connection': 'keep-alive', 'x-amzn-requestid': 'fd46a5c9-0353-494a-a5d9-d0b43fa5b698'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': '9be3c754-d792-4b9a-80e4-988130797bb1'}, {'itemId': '8ef855f7-8746-423d-b960-5cea2c6f4ecb'}, {'itemId': 'd4caa74c-5d27-4f1a-847d-e2d93a6fdf32'}, {'itemId': '17098976-d5cc-4470-b4b4-bdaed34ae6a0'}, {'itemId': '8dc592e0-b192-44d7-a639-b79946f4e12b'}, {'itemId': '62259467-5d98-4c6d-a4f2-72e57bf52da6'}, {'itemId': '9fcced83-5621-4c3c-88dd-f3110360c47e'}, {'itemId': '82148184-2e41-471b-a0d7-e58caa2333e9'}, {'itemId': 'a56308af-abd2-41a0-9cf3-b4a040fd8d3f'}, {'itemId': 'e6c07e26-3f21-4b47-9f78-0afd24dfa409'}, {'itemId': 'de374b6b-7636-4784-b555-b5b37ee158e0'}, {'itemId': 'f6275b6c-1a1c-4122-8fd0-048043f9c871'}, {'itemId': 'af0290e7-3bc3-493f-b1d7-

Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for products render in a nice dataframe. Again, let's create a helper function to achieve this.

In [46]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_viewed_x(recommendations_df, item_id, user_id):
    # Get the item name
    original_item_name = get_item_name_from_id(item_id)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_customers_who_viewed_arn,
        itemId = str(item_id),
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [original_item_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different items. Let's sample some data from our dataset to test our "Customers who viewed X also viewed" Recommender. Grab 5 random items from our dataframe.

In [47]:
samples = item_metadata_df.sample(5)
samples

,id,name,category,style,featured
125,101c0296-48b4-496d-accd-2205e79a0048,Stylish Glasses for Bold Women,accessories,glasses,NaN
1223,fe01f22e-3783-4e27-8ae5-e2cd0cbcaf61,Irresistible Vanilla Buttercream Dream Cake,groceries,bakery,NaN
42,8b93bc57-62ca-4c6f-a83a-c527da84f224,Rustic Explorer Backpack,accessories,backpack,NaN
1555,304cd58e-1e7e-4151-9716-1bb222cb724e,Stylish Linen Floor Lamp,homedecor,lighting,NaN
487,1e58593d-6395-41ca-9fb0-a9607337ce18,Gentle Toothbrush for Healthier Smiles,beauty,grooming,NaN


In [48]:
viewed_x_also_viewed_recommendations_df = pd.DataFrame()
items = samples.id.tolist()

for item in items:
    viewed_x_also_viewed_recommendations_df = get_new_recommendations_df_viewed_x(viewed_x_also_viewed_recommendations_df, item, user_id)

viewed_x_also_viewed_recommendations_df

,Stylish Glasses for Bold Women,Irresistible Vanilla Buttercream Dream Cake,Rustic Explorer Backpack,Stylish Linen Floor Lamp,Gentle Toothbrush for Healthier Smiles
0,Chic Glasses Elevate Your Look,Tender Briny Oysters Ready to Savor,Stylish Reversible Paisley Belt,Bronze Lamp Illuminates with Elegance,Tranquil Bath Oil Escape
1,Stylish Beige Leather Handbag,Freshly Baked Artisan Bread,Fun Print Stretch Belt,Soft Tan Square Accent Cushion,Soft Brush for Flawless Lips
2,Sleek Polarized Glasses for Her,Creamy Vanilla Cake - A Sweet Indulgence,Stylish Cinched Waist Belt,Stylish Gray Cushion Softens Any Space,Festive Wreath with Fresh Berries
3,Rugged Leather Backpack,Stylish Faux Leather Backpack,Stylish Leather Belt Elevates Any Outfit,Charming Handcrafted Easter Basket Decor,Refresh Skin with Hydrating Beauty Lotion
4,Stylish Leather Belt for Her,Fresh-Caught Sustainable Jumbo Shrimp,Rugged Leather Backpack,Sleek Curved Vase Elevates Any Decor,Funky Retro Beaded Belt
5,Stylish Tan Handbag,Stylish Cat-Eye Glasses for Her,"Sleek Leather Belt, Sophisticated Style",Natural Wax Taper - Elegant Home Lighting,Radiant Perfection Concealer
6,Stylish Smartwatch for Active Women,Stylish Faux Leather Belt,Sleek Black Leather Handbag,Stylish Gray Shirt Dress,Luxurious Fragrant Soap for Silky Skin
7,Scrub Away Dull Skin,"Crimson Onions, Versatile Staple",Sparkling Crystal Watch for Her,Sleek Modern Alarm to Brighten Your Mornings,Stylish Retro Glasses for Any Look
8,Fluffy Frosted Vanilla Cupcakes,Juicy Crunchy Fresh Apples,Sleek Waist-Cinching Belt for Polished Style,Sleek Modern Wall Clock in White,Discrete On-the-Go Makeup Touch-Ups
9,Juicy Fresh Red Tomatoes,Sweet Surprise Easter Egg,Spacious Tan Backpack for Her Travels,Stylish Gray Cushion Set,Indulgent Floral Soap for Pampering


Based on the random product selected above, do the similar item recommendations from Personalize make sense? Keep in mind that the "Customers who viewed X also viewed" recommendations are based on the interactions we generated as input into the solution creation process above.

You may notice that some items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with a small dataset).

### "Recommended for you" Recommender

"Recommended for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

With this use case, Amazon Personalize automatically filters items the user purchased based on the `userId` that you specify and "Purchase" events. For better performance, include "Purchase" events along with the required "View" events. 

In [49]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20
)

In [50]:
item_list = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)


,Item,Name,Category,Style
0,b4e0ec0f-efec-4103-a741-d9e373808130,Sleek Electric Guitar - Play Like the Pros,instruments,strings
1,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone",instruments,percussion
2,3999c48f-4ccf-463b-8e0b-f28dcdc469d1,"Majestic Grand Piano, Powerfully Resonant",instruments,keys
3,baae9ca1-2fd5-4161-8eb3-d0c0616e34f6,Adventure Awaits with Pet Outdoors Gear,outdoors,pet
4,aafaef40-a269-4d41-a813-e9bb95ce6aa7,Insulated Camp Cup for Outdoor Adventure,outdoors,camping
5,007bf018-a074-44df-80d7-70866f7bead8,Portable LED Lantern - Light Your Adventures,outdoors,camping
6,dc135daa-4c1f-4672-ada4-fc8ee144e2f1,Portable Bright Camp Lamp,outdoors,camping
7,579ca23a-2d8e-4229-812e-f68665b3a838,Brighten Your Campsite,outdoors,camping
8,1b997958-e55c-4e1d-9764-ae969504b94d,Adventure Awaits in this Sleek Kayak,outdoors,kayaking
9,5106afab-048f-4d9f-b62b-3166e42ba01e,Leakproof Camping Water Bottle,outdoors,camping


Since "Recommended for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [51]:
samples = user_metadata_df.sample(3)
samples

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
2748,2749,True,M,Robert,Nguyen,robert.nguyen@example.com,47,Robert Nguyen,user2749,homedecor_electronics_outdoors,lower_priced_products,{},{'ios': {'anonymous_id': 'e654dd4a-71c3-4222-b...,"[{'first_name': 'Robert', 'last_name': 'Nguyen..."
500,501,True,M,Andrew,Park,andrew.park@example.com,42,Andrew Park,user501,accessories_groceries_books,lower_priced_products,{},{'ios': {'anonymous_id': '66bd483a-2271-43be-8...,"[{'first_name': 'Andrew', 'last_name': 'Park',..."
2218,2219,True,F,Ashley,Foster,ashley.foster@example.com,50,Ashley Foster,user2219,instruments_books_electronics,discount_indifferent,{},{'ios': {'anonymous_id': '9048d8d1-bd31-41e4-a...,"[{'first_name': 'Ashley', 'last_name': 'Foster..."


Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Recommended for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a product.

The cells below will handle getting recommendations from the "Recommended for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [52]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_recommended_for_you(recommendations_df, user_id):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [53]:
recommendations_df_users = pd.DataFrame()
users = samples.id.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_recommended_for_you(recommendations_df_users, user)

recommendations_df_users

,2749,501,2219
0,Vivid Moments Captured Easily,Stylish Leather Handbag,"Crisp Wood Tones, Naturally"
1,Immersive Audio Anywhere Headphones,Stylish Glasses for Sharp Dressed Men,Resonant Acoustic Bass - Deep Tones
2,Sleek Adjustable Reading Floor Lamp,Stylish Hip Glasses for Men,Bright Zildjian Cymbals with Balanced Tone
3,Bronze Lamp Illuminates with Elegance,Stylish Reading Glasses for Men,Resonant Acoustic Drum Kit
4,Stylish White Floor Lamp,Sleek Black Backpack,Sleek Maple Bass Guitar
5,Stylish Linen Floor Lamp,Stylish Sophisticated Eyewear for Confident Men,Unmatched Sound for Musicians
6,Stylish Bronze Lamp Brightens Any Room,Stylish Leather Handbag for Daily Use,Smooth Acoustic Bass for Deep Tones
7,Sleek White Minimalist Wall Clock,Stylish Blue Backpack - Store in Style,Resonant Mahogany Acoustic Drum Kit
8,Sleek Compact Keyboard for Fast Typing,Sleek Black Backpack for Urban Adventure,Discover Scenic Beauty
9,Stylish Gray Cushion Set,Stylish Glasses for Men,Innovative Acoustic Drum Kit


## Applying the discount context

We'll get the user recommendations when discount context is applied for comparison. This is using the "contextual metadata" feature of Amazon Personalize. In this case, our context is whether a discount was applied.

In [54]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20,
    context = {'DISCOUNT':'Yes'} # Here we provide the context for the recommendations
)

item_list_context = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list_context ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list_context ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

,Item,Name,Category,Style
0,b4e0ec0f-efec-4103-a741-d9e373808130,Sleek Electric Guitar - Play Like the Pros,instruments,strings
1,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone",instruments,percussion
2,3999c48f-4ccf-463b-8e0b-f28dcdc469d1,"Majestic Grand Piano, Powerfully Resonant",instruments,keys
3,baae9ca1-2fd5-4161-8eb3-d0c0616e34f6,Adventure Awaits with Pet Outdoors Gear,outdoors,pet
4,aafaef40-a269-4d41-a813-e9bb95ce6aa7,Insulated Camp Cup for Outdoor Adventure,outdoors,camping
5,007bf018-a074-44df-80d7-70866f7bead8,Portable LED Lantern - Light Your Adventures,outdoors,camping
6,579ca23a-2d8e-4229-812e-f68665b3a838,Brighten Your Campsite,outdoors,camping
7,dc135daa-4c1f-4672-ada4-fc8ee144e2f1,Portable Bright Camp Lamp,outdoors,camping
8,1b997958-e55c-4e1d-9764-ae969504b94d,Adventure Awaits in this Sleek Kayak,outdoors,kayaking
9,54dc5dd9-3df4-410a-ae55-54844988a0ca,Adventure On with Man's Best Friend,outdoors,pet


Let us compare it to the previous set of recommendations without context.

In [55]:
df = pd.DataFrame()
df['Item (No context)'] = [ itm['itemId'] for itm in item_list ]
df['Name (No context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]

df['Item (Context)'] = [ itm['itemId'] for itm in item_list_context ]
df['Name (Context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

,Item (No context),Name (No context),Item (Context),Name (Context)
0,b4e0ec0f-efec-4103-a741-d9e373808130,Sleek Electric Guitar - Play Like the Pros,b4e0ec0f-efec-4103-a741-d9e373808130,Sleek Electric Guitar - Play Like the Pros
1,7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone",7b8e35ec-ddb6-465a-b209-0a6bf8ec1bc9,"Resonant Acoustic Drum, Warm Tone"
2,3999c48f-4ccf-463b-8e0b-f28dcdc469d1,"Majestic Grand Piano, Powerfully Resonant",3999c48f-4ccf-463b-8e0b-f28dcdc469d1,"Majestic Grand Piano, Powerfully Resonant"
3,baae9ca1-2fd5-4161-8eb3-d0c0616e34f6,Adventure Awaits with Pet Outdoors Gear,baae9ca1-2fd5-4161-8eb3-d0c0616e34f6,Adventure Awaits with Pet Outdoors Gear
4,aafaef40-a269-4d41-a813-e9bb95ce6aa7,Insulated Camp Cup for Outdoor Adventure,aafaef40-a269-4d41-a813-e9bb95ce6aa7,Insulated Camp Cup for Outdoor Adventure
5,007bf018-a074-44df-80d7-70866f7bead8,Portable LED Lantern - Light Your Adventures,007bf018-a074-44df-80d7-70866f7bead8,Portable LED Lantern - Light Your Adventures
6,dc135daa-4c1f-4672-ada4-fc8ee144e2f1,Portable Bright Camp Lamp,579ca23a-2d8e-4229-812e-f68665b3a838,Brighten Your Campsite
7,579ca23a-2d8e-4229-812e-f68665b3a838,Brighten Your Campsite,dc135daa-4c1f-4672-ada4-fc8ee144e2f1,Portable Bright Camp Lamp
8,1b997958-e55c-4e1d-9764-ae969504b94d,Adventure Awaits in this Sleek Kayak,1b997958-e55c-4e1d-9764-ae969504b94d,Adventure Awaits in this Sleek Kayak
9,5106afab-048f-4d9f-b62b-3166e42ba01e,Leakproof Camping Water Bottle,54dc5dd9-3df4-410a-ae55-54844988a0ca,Adventure On with Man's Best Friend


## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For an ECOMMERCE application you want to dynamically render a personalized shelf/rail/carousel based on some information (category, style, season, etc…). This may not be information that you have in your metadata, so an item metadata filter will not work, however you may have this information within your system to generate the item list. You can use this campaign to rank the products listed for each category and the featured products list, as well as ranking catalog search results displayed in a search widget.

To demonstrate this, we will use the same user from before and a random collection of items.

#### Get Featured Products List

First let's get the list of featured products from the Products data.

In [56]:
featured_products = item_metadata_df[item_metadata_df['featured']==True]

#### Rank Featured Products

Using the featured products list just retrieved, first we'll create a list of item IDs that we want to rank for a specific user. This ranking allows us to provide ranked products based on the user's behavior. These behaviors should be consistent to the same persona that was mentioned above (since we're going to use the same `user_id`).

In [57]:
unranked_product_ids = list (featured_products['id'])

df = pd.DataFrame()
df['Item'] = [ itm for itm in unranked_product_ids ]
df['Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]
df['Category'] = [ get_item_category_from_id ( itm) for itm in unranked_product_ids ]
df['Style'] = [ get_item_style_from_id ( itm) for itm in unranked_product_ids ]
display (df)

,Item,Name,Category,Style
0,2b67230f-dc22-462e-9afe-c9e459f74093,Earthy Tan Handbag for Free Spirits,accessories,handbag
1,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,apparel,jacket
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,beauty,bathing
3,5d37a44b-d121-426e-b528-59e603ba5923,Egypt Travel Guide,books,travel
4,3b145528-d5fc-4c2a-b2a5-e119128caa5f,"Block Outside Noise, Hear Pure Sound",electronics,headphones
5,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,floral,bouquet
6,22552eb1-57f1-4fa3-a93a-a9fa22851f9f,Bold Red Sneakers for All-Day Comfort,footwear,sneaker
7,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,furniture,sofas
8,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Nature's Vitamin C,groceries,fruits
9,01a8978b-2a84-4dbd-acc4-aff74a468681,Stylish Curved Ceramic Vase,homedecor,decorative


Now let's have Personalize rank the featured product IDs based on our random user.

In [58]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

[
    {
        "itemId": "3f9a39b2-0d63-4751-b6ee-4ecd08dd2276",
        "score": 0.6315805
    },
    {
        "itemId": "7160b264-e3ed-4ac3-9dd7-2c537b00e5ed",
        "score": 0.348232
    },
    {
        "itemId": "a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec",
        "score": 0.0069587
    },
    {
        "itemId": "8bffb5fb-624f-48a8-a99f-b8e9c64bbe29",
        "score": 0.005983
    },
    {
        "itemId": "5d37a44b-d121-426e-b528-59e603ba5923",
        "score": 0.003093
    },
    {
        "itemId": "2b67230f-dc22-462e-9afe-c9e459f74093",
        "score": 0.0022739
    },
    {
        "itemId": "3b145528-d5fc-4c2a-b2a5-e119128caa5f",
        "score": 0.0009746
    },
    {
        "itemId": "b87da3f8-9a3e-417d-abd7-16329c5be1ba",
        "score": 0.0006158
    },
    {
        "itemId": "6f04daee-7387-442f-bc99-a9b0072b29ce",
        "score": 0.0001388
    },
    {
        "itemId": "6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b",
        "score": 3.89e-05
    },
    {
        "itemId"

We will add the ranked items as a second column to the original dataframe, for a side-by-side comparison.

In [59]:
df = pd.DataFrame()
df['Original List'] = [ itm for itm in unranked_product_ids]
df['Original List Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]

df['Reranking'] = [ itm['itemId'] for itm in reranked]
df['Reranking Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in reranked ]

display (df)

,Original List,Original List Name,Reranking,Reranking Name
0,2b67230f-dc22-462e-9afe-c9e459f74093,Earthy Tan Handbag for Free Spirits,3f9a39b2-0d63-4751-b6ee-4ecd08dd2276,Vibrant Tone Electric Guitar
1,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Fetch-tastic Frisbee for Playful Pups
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Nature's Vitamin C
3,5d37a44b-d121-426e-b528-59e603ba5923,Egypt Travel Guide,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Durable Screwdriver for Any Task
4,3b145528-d5fc-4c2a-b2a5-e119128caa5f,"Block Outside Noise, Hear Pure Sound",5d37a44b-d121-426e-b528-59e603ba5923,Egypt Travel Guide
5,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,2b67230f-dc22-462e-9afe-c9e459f74093,Earthy Tan Handbag for Free Spirits
6,22552eb1-57f1-4fa3-a93a-a9fa22851f9f,Bold Red Sneakers for All-Day Comfort,3b145528-d5fc-4c2a-b2a5-e119128caa5f,"Block Outside Noise, Hear Pure Sound"
7,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin
8,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Nature's Vitamin C,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Pumpkin Lights for Halloween
9,01a8978b-2a84-4dbd-acc4-aff74a468681,Stylish Curved Ceramic Vase,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user that cannot be easily categorized in your metadata, for instance "featured products" which are curated by a person.


# Filters <a class="anchor" id="filters"></a>

## Create Filters <a class="anchor" id="create-filters"></a>
[Back to top](#top)

Personalize can utilize either [static or dynamic filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html). Static filters are where the filter properties are built into the filter itself, which makes invocation simpler, but gives less flexibility. An example of this would be an accessories category filter, which invokes the get_recommendations_response api with the specific filter of CATEGORY_L1 = accessories. In order to create a recommendation for each filter, that would require 10+ filters. Personalize also supports dynamic filters, where the values can be passed at runtime, allowing for a single filter of CATEGORY_L1, where the actual category is passed at runtime. 

Filters can be created for fields of both Items and Events. 

A few common use cases for dynamic filters in ECOMMERCE are:

* Categorical filters based on Item Metadata - Often your item metadata will have information about the item. Filtering on these can provide recommendations within that data.

* Range based filters based on Item Metadata - Personalize supports range operations in both static and dynamic filters. Filtering based on a range can be used to create recommendations such as "Whats on sale now".

* Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "recommended for you" recommendation to a "purchase again" recommendations.



### Create Category Filter

Since there are a lot of categories to filter on, we will create a dynamic filter using the dynamic variable $CATEGORY, this will allow us to pass in the variable at runtime rather than create a static filter for each category.

In [60]:
category_filter_name = 'filter-category'

try:
    create_category_filter_response = personalize.create_filter(
        name = category_filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'INCLUDE ItemID WHERE Items.CATEGORY_L1 IN ($CATEGORY)'
    )
    
    category_filter_arn = create_category_filter_response['filterArn']
    print('Creating the Personalize filter with category_filter_arn {}.'.format(category_filter_arn))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    category_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+category_filter_name
    print('The Personalize filter {} already exists.'.format(category_filter_name))
    print ('\nWe will be using the existing Personalize Filter with category_filter_arn = {}'.format(category_filter_arn))
    

The Personalize filter filter-category already exists.

We will be using the existing Personalize Filter with category_filter_arn = arn:aws:personalize:us-east-1:381491864570:filter/filter-category


### Create Purchased Products Filter

Since it's a poor user experience to recommend products that a user has already purchased, we will create a filter that excludes recently purchased products. We'll do this by creating a filter expression that excludes items that have an interaction with an event type of `Purchase` for the user. If you recall from the last notebook, the interactions dataset uses the `Purchase` event type to track purchased items.

In [61]:
filter_name = 'filter-purchased-products'

try:
    create_purchase_filter_response = personalize.create_filter(
        name=filter_name,
        datasetGroupArn = workshop_dataset_group_arn,
        filterExpression = 'EXCLUDE itemId WHERE INTERACTIONS.event_type in ("Purchase")'
    )
    
    purchase_filter_arn = create_purchase_filter_response['filterArn']

    print('Creating the Personalize filter with purchase_filter_arn {}.'.format(purchase_filter_arn))
    
except personalize.exceptions.ResourceAlreadyExistsException as e:
    purchase_filter_arn = 'arn:aws:personalize:'+region+':'+account_id+':filter/'+filter_name
    print('The Personalize filter {} already exists.'.format(filter_name))
    print ('\nWe will be using the existing Personalize Filter with purchase_filter_arn = {}'.format(purchase_filter_arn))
    

The Personalize filter filter-purchased-products already exists.

We will be using the existing Personalize Filter with purchase_filter_arn = arn:aws:personalize:us-east-1:381491864570:filter/filter-purchased-products


### Wait for Filter Status to Become ACTIVE

The filter should take a minute or so to become active.

In [62]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = purchase_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Purchase filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)
    
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = category_filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Category Filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

Purchase filter: ACTIVE
Category Filter: ACTIVE


## Using Filters <a class="anchor" id="using-filters"></a>
[Back to top](#top)

Now that the Filters have been created we can use them to filter our recommendations.

## Category Filter<a class="anchor" id="filter">
[Back to top](#top)

To test our purchased products filter, we will request recommendations for a random user with a particular category. 
    
We will create a helper function to return recommendations using a dynamic flilter:

In [63]:
def get_new_recommendations_df_by_dynamic_filter(recommendations_df, user_id, category_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_recommended_for_you_arn,
        userId = str(user_id),
        filterArn = category_filter_arn,
        filterValues = { "CATEGORY": "\"" + filter_values + "\""},
        numResults = 10
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        product = get_item_name_from_id(item['itemId'])
        recommendation_list.append(product)
    filter_name = category_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

We will select the first 7 categories for this example.

In [64]:
unique_category_field_values = list(item_metadata_df['category'].unique()[:7])
unique_category_field_values

['accessories',
 'apparel',
 'beauty',
 'books',
 'electronics',
 'floral',
 'footwear']

In [65]:
# Iterate through Categories
recommendations_df_category = pd.DataFrame()
for category in unique_category_field_values:
    recommendations_df_category = get_new_recommendations_df_by_dynamic_filter(recommendations_df_category, user_id, category_filter_arn , category)
    
recommendations_df_category

,accessories,apparel,beauty,books,electronics,floral,footwear
0,Rustic Tan Backpack for Adventure,Stylish Slate Gray Women's Insulated Jacket,Tranquil Bath Oil Escape,Spicy Flavors of Indonesian Cuisine,Durable Cable Connects Your World,Lush Cascading Air-Purifying Plant,Stylish Leather Sandals for Her
1,Sleek Gray Travel Backpack,Funky Retro Color Socks,Indulgent Shea & Cocoa Beauty Soap,Italian Flavors Cookbook,Audiophile Bliss Headphones,Vibrant Floral Bouquet Brightens Your Day,Trendy White Leather Sneakers
2,Stylish Black Crossbody Bag,Cozy Firebrick Women's Insulated Jacket,Soft Lips with Shea,Egypt Travel Guide,Immersive Audio Anywhere Headphones,Breathtaking Organic Wedding Bouquet,Stylish Faux Leather Platform Heels
3,Slate Pack - Roomy & Refined,Navy Casual Jacket - Cozy All-Season Style,Scrub Away Dull Skin,A Taste of Turkey,Fast Desktop Computer for Work and Play,Vibrant Seasonal Wedding Bouquet,Stylish Dark Red Leather Shoes
4,Rustic Leather Tote,Stylish Blue Jacket for Outdoor Women,"Soft Bristles, Bright Smiles",Ancient Secrets of Mexico Travel Guide,Relive Special Moments in HD,Vibrant Carnation Floral Bouquet,Fun Floral Sandals
5,Stylish Leather Handbag,Tranquil Sea Print Relaxed Tee,Velvety Soft Luxury Bath Towel,French Cuisine Mastery in Your Kitchen,Powerful Desktop for Work and Play,Lush Green Plant Brightens Your Space,Sleek Black Wedge Heels
6,Sleek Black Travel Backpack,Stylish Black Shirt for Women,Silky Smooth Skin Massage Oil,Spicy Thai Cuisine Cookbook,"Sleek Sound, Stylish Design",Crimson Cascade Rose Bouquet,Stylish Black Leather Ankle Boots
7,Sleek Dark Gray Travel Backpack,Cheery Yellow-Green Insulated Jacket,Plush Cotton Bath Towel,A Taste of Spanish Wonders,Sleek Connectivity Cable for Charging and Tran...,Velvety Rose Bouquet,Sleek Black Heels Elevate Your Style
8,Stylish Dark Gray Handbag,Stylish Olive Jacket for Women,Luxurious Cotton Towel for Pampering,Spicy Thai Home Cooking Cookbook,Speedy Desktop PC for Work and Play,Vibrant Red Tulip Floral Arrangement,Stylish Crimson Shoes for Women
9,Rustic Explorer Backpack,Vibrant Boho Scarf,Revitalizing Organic Everyday Soap,Vivid Japan Travel Adventures,Powerful Speakers - Experience Immersive Sound,Bright White Lilies Bouquet,Red Formal Heels for Special Style


As an alternative to some of the steps above we could also have used the return item metadata functionality of personalize.

<div class="alert alert-block alert-warning">
<b>Important:</b> When you enable metadata in recommendations, you incur additional costs. For more information see Amazon Personalize pricing: https://aws.amazon.com/personalize/pricing/.
</div>

In [79]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_recommended_for_you_arn,
    userId = str(user_id),
    numResults = 5,
    metadataColumns = {
            "ITEMS": ["PRODUCT_DESCRIPTION", "PRICE"]
        }
)

get_recommendations_response

{'ResponseMetadata': {'RequestId': '35b4b168-d9e1-4ce9-830a-4c033bf3a024',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 27 Apr 2024 20:38:14 GMT',
   'content-type': 'application/json',
   'content-length': '2095',
   'connection': 'keep-alive',
   'x-amzn-requestid': '35b4b168-d9e1-4ce9-830a-4c033bf3a024'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': 'cfafd627-7d6b-43a5-be05-4c7937be417d',
   'metadata': {'price': '58.0',
    'product_description': "Expertly crafted for precision, this versatile Chef's Knife features a durable stainless steel blade perfect for chopping, slicing, and dicing. An essential tool for any home cook."}},
  {'itemId': '00740ca5-372b-4e72-a040-8eacde2ecf4f',
   'metadata': {'price': '69.0',
    'product_description': 'Spread joy on every slice with our essential Butter Knife. Specifically designed for smooth, even spreading without ripping breads or pastries, this versatile kitchen tool makes every bite better. A must-have for bakers and home c

## Real-time Events<a class="anchor" id="event-tracking"></a>
[Back to top](#top)

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in retail workloads, because what products the user is interested in now may be different than what they were interested in a week ago, a day ago, or even a few minutes ago.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

Start by creating an event tracker that is attached to the dataset group. This event tracker will add information to the dataset and will influence the recommendations.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [66]:
event_tracker_name = 'personalize-poc-event-tracker'
try: 
    create_event_tracker_response = personalize.create_event_tracker(
        name = event_tracker_name,
        datasetGroupArn=workshop_dataset_group_arn
        )
    event_tracker_arn = create_event_tracker_response['eventTrackerArn']
    print(json.dumps(create_event_tracker_response, indent=2))
    print ('\nCreating the event_tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    tracking_id = create_event_tracker_response['trackingId']
    print ('\nAnd trackingId = {}'.format(tracking_id))
    

except personalize.exceptions.ResourceAlreadyExistsException as e:
    event_tracker_list = personalize.list_event_trackers( 
        datasetGroupArn= workshop_dataset_group_arn
    )['eventTrackers']
    
    event_tracker_arn = event_tracker_list[0]['eventTrackerArn']
    
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    tracking_id = describe_event_tracker_response['eventTracker']['trackingId']
    
    print ('\nThe the Event Tracker with event_tracker_name {} already exists'.format(event_tracker_name))
    print ('\nWe will be using the existing Event Tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    print ('\nAnd tracking_id = {}'.format(tracking_id))
    


The the Event Tracker with event_tracker_name personalize-poc-event-tracker already exists

We will be using the existing Event Tracker with event_tracker_arn = arn:aws:personalize:us-east-1:381491864570:event-tracker/f22394de

And tracking_id = 85cf39d4-d2c5-40a2-bf21-f4678366f151


### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [67]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

EventTracker: ACTIVE


### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker. Since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. This challenge can be solved by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows applications to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, client applications can pass the provisional user ID to the user management system when creating a new user account. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

### Using the Purchased Products Filter

To use purchased products filter, we will request ranked recommendations for a random user. Then we will send a `Purchase` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter. The purchased product would be excluded from the new recommendations.

In [68]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = 456
display(user_metadata_df[user_metadata_df['id']==user_id])

,id,selectable_user,gender,first_name,last_name,email,age,name,username,persona,discount_persona,traits,platforms,addresses
455,456,True,M,Austin,Jensen,austin.jensen@example.com,35,Austin Jensen,user456,housewares_floral_seasonal,all_discounts,{},{'ios': {'anonymous_id': '1212fae7-a8eb-4709-b...,"[{'first_name': 'Austin', 'last_name': 'Jensen..."


In [69]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id)
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

,Item,Name,Category,Style
0,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,floral,bouquet
1,1e96e374-be23-4c97-b87e-b5c45cb8999f,Sleek Stainless Steel Pots and Pans,housewares,kitchen
2,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,beauty,bathing
3,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Durable Screwdriver for Any Task,tools,screwdriver
4,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Pumpkin Lights for Halloween,seasonal,halloween
5,a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec,Nature's Vitamin C,groceries,fruits
6,2ad09e8e-fd41-4d29-953e-546b924d7cb8,Sparkling Pendant Necklace,jewelry,necklace
7,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,furniture,sofas
8,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,apparel,jacket
9,3f9a39b2-0d63-4751-b6ee-4ecd08dd2276,Vibrant Tone Electric Guitar,instruments,strings


Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [70]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')
print(f'Product name: {get_item_name_from_id ( product_id_to_purchase)}')

Product to simulate purchasing: a31ad4b3-f9a8-4a9b-a8b3-3034af7bacec
Product name: Nature's Vitamin C


Next, let's send a `Purchase` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [71]:
response = personalize_events.put_events(
    trackingId = tracking_id,
    userId = str(user_id),
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'Purchase',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for Purchase event to become consistent.
time.sleep(10)

print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "2dbaa545-95a5-4dae-9e6b-ea8804515944",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 27 Apr 2024 20:33:00 GMT",
      "content-type": "application/json",
      "content-length": "0",
      "connection": "keep-alive",
      "x-amzn-requestid": "2dbaa545-95a5-4dae-9e6b-ea8804515944"
    },
    "RetryAttempts": 0
  }
}


Finally, let's retrieve recommendations for the user again, but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.

In [72]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = workshop_rerank_campaign_arn,
    inputList = unranked_product_ids,
    userId = str(user_id),
    filterArn = purchase_filter_arn
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

,Item,Name,Category,Style
0,4bb66b8a-cf13-4959-87ce-ca506fa568a2,Breathtaking Wedding Bouquet of Flowers,floral,bouquet
1,b87da3f8-9a3e-417d-abd7-16329c5be1ba,Luxurious Fragrant Soap for Silky Skin,beauty,bathing
2,8bffb5fb-624f-48a8-a99f-b8e9c64bbe29,Durable Screwdriver for Any Task,tools,screwdriver
3,6f04daee-7387-442f-bc99-a9b0072b29ce,Spooky Pumpkin Lights for Halloween,seasonal,halloween
4,2ad09e8e-fd41-4d29-953e-546b924d7cb8,Sparkling Pendant Necklace,jewelry,necklace
5,8b9733b9-cbea-4de3-978b-5e3f0e8c796c,Indulge in Luxurious Leather Comfort,furniture,sofas
6,6bd74f2d-90c0-4ca6-9663-f3bbe9bf405b,Sleek Dark Red Men's Jacket,apparel,jacket
7,3f9a39b2-0d63-4751-b6ee-4ecd08dd2276,Vibrant Tone Electric Guitar,instruments,strings
8,01a8978b-2a84-4dbd-acc4-aff74a468681,Stylish Curved Ceramic Vase,homedecor,decorative
9,7160b264-e3ed-4ac3-9dd7-2c537b00e5ed,Fetch-tastic Frisbee for Playful Pups,outdoors,pet


The following code will raise an assertion error if the product we just purchased is still recommended.

In [73]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

Purchased item filtered from recommendations for user!


In [39]:
%store workshop_dataset_group_arn
%store region
%store role_name
%store s3_access_policy_arn

# Variables required for Optional Notebook 1 - Batch Inference
%store users
%store unranked_product_ids
%store bucket_name
%store workshop_rerank_solution_version_arn
%store item_metadata_df

Stored 'workshop_dataset_group_arn' (str)
Stored 'region' (str)
Stored 'role_name' (str)
Stored 's3_access_policy_arn' (str)
Stored 'users' (list)
Stored 'unranked_product_ids' (list)
Stored 'bucket_name' (str)
Stored 'workshop_rerank_solution_version_arn' (str)
Stored 'item_metadata_df' (DataFrame)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`Retail_04_Clean_Up.ipynb`](Retail_04_Clean_Up.ipynb).

You can choose to head to any of the optional notebooks in this folder to continue experimenting with Amazon Personalize.

<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: Run optional notebooks BEFORE you run the clean-up notebook.
</div>
